In [ ]:
# Import libraries
import matplotlib.pyplot as plt 
import numpy as np 
import os
import pandas as pd
import yaml

In [ ]:
# Load payoff matrices
red, blue = np.load(snakemake.input.red), np.load(snakemake.input.blue)

# Load df
eq_df = pd.read_csv(snakemake.input.equilibria_csv,index_col=0).sort_index()
eq_arr = np.load(snakemake.input.equilibria_arr)


In [ ]:
# Calculate welfare
X = red + blue

# Calculate best outcome
X_max = np.max(X)
X_max

In [ ]:
def imshow(X,ax,fontsize=3.0,cmap="viridis"):
    '''Labels the image
    '''
    ax.imshow(X,cmap=cmap)
    
    for i, row in enumerate(X):
        for j, val in enumerate(row):
            text = ax.text(j,i,f"{X[i,j]:0.2f}",
                           fontsize=fontsize,
                           ha="center",
                           va="center",
                           color="w")

# Get Stats from worst Nash Equilibrium
idx = eq_df['welfare'].idxmin()
eq_stats = eq_df.loc[idx]
eq = eq_arr[idx]

# Calculate poa
eq_stats['poa'] = X_max / eq_stats['welfare']

with open(snakemake.output[0],'w') as file:
    yaml.dump(eq_stats.to_dict(),file,default_flow_style=False)

# Plot reward
fig = plt.figure(figsize=(3,3),dpi=300)

gs = fig.add_gridspec(2,2,
                      width_ratios=(4,1),
                      height_ratios=(1,4),
                      left=0.1, right=0.9,
                      bottom=0.1, top=0.9,
                      wspace=0.05,
                      hspace=0.05)

ax = fig.add_subplot(gs[1,0])
imshow(X,ax)

x = np.arange(eq.shape[1])
ax_col_player = fig.add_subplot(gs[0,0], sharex=ax)
ax_col_player.bar(x,eq[0])
ax_col_player.set_ylim([0,1])
ax_col_player.set_xticks([])
    
ax_row_player = fig.add_subplot(gs[1,1], sharey=ax)
ax_row_player.barh(x,eq[1])
ax_row_player.set_yticks([])
ax_row_player.set_xlim([0,1])


plt.savefig(snakemake.output[1],bbox_inches="tight")

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(12,6))

imshow(red,axes[0],fontsize=6,cmap="Blues")
imshow(blue,axes[1],fontsize=6,cmap="Reds")

plt.savefig(snakemake.output[2])